In [1]:
from tensorflow.python.keras.applications.mobilenet import MobileNet
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.applications.mobilenet import preprocess_input, decode_predictions
import numpy as np
import tensorflow as tf
import urllib.request as urllib
import functools
import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()

/opt/conda/envs/riptide-py3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.python.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.map(lambda data, label: (tf.cast(data, tf.float32)/255, tf.cast(label, tf.int32)))
train_dataset = train_dataset.batch(64).repeat(1).shuffle(1000)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.map(lambda data, label: (tf.cast(data, tf.float32)/255, tf.cast(label, tf.int32)))
test_dataset = test_dataset.batch(64).repeat(1)

In [15]:
class MNISTModel(tf.keras.Model):
  def __init__(self):
    super(MNISTModel, self).__init__()
    self.dense1 = tf.keras.layers.Dense(units=64, activation='relu')
    self.dense2 = tf.keras.layers.Dense(units=128, activation='relu')
    self.dense3 = tf.keras.layers.Dense(units=128, activation='relu')
    self.dense4 = tf.keras.layers.Dense(units=10)
    self.flatten = tf.keras.layers.Flatten()

  def call(self, input):
    """Run the model."""
    result = self.flatten(input)
    result = self.dense1(result)
    result = self.dense2(result)
    result = self.dense3(result)
    result = self.dense4(result)  # reuse variables from dense2 layer
    return result

model = MNISTModel()

In [16]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)

In [17]:
for i, (data, label) in enumerate(tfe.Iterator(train_dataset)):
    with tfe.GradientTape() as tape:
        prediction = model(data)
        loss = tf.losses.sparse_softmax_cross_entropy(labels=label, logits=prediction)
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(zip(grads, model.variables), global_step=tf.train.get_or_create_global_step())
    if i % 100 == 0:
        print("Loss at step {:04d}: {:.3f}".format(i, loss))

Loss at step 0000: 2.343
Loss at step 0100: 2.193
Loss at step 0200: 2.031
Loss at step 0300: 1.903
Loss at step 0400: 1.882
Loss at step 0500: 1.739
Loss at step 0600: 1.929
Loss at step 0700: 1.747
